In [1]:
import pandas as pd # dataframes
import numpy as np # maths
import matplotlib.pyplot as plt # plots
import seaborn as sns # nicer plots

from sklearn.model_selection import train_test_split # splitting dataframes and subsplits
import time # model compuational expense
from numpy import array # use for getting one hot codings
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # get binary outcomes
from sklearn import ensemble # models random forest and boosted tree

# evaluation metrics
from sklearn.metrics import classification_report # f1 scores
from sklearn.metrics import confusion_matrix # prediction vs actual
from sklearn.metrics import accuracy_score # overall score
from sklearn.model_selection import cross_val_score # check for overfitting and model stability

sns.set(style="darkgrid") # plotting if any
# plot formatting
%matplotlib inline

In [2]:
# get dataset and take a look
df = pd.read_csv('train.csv')
df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [3]:
# one hot encoding for outcome variables
data = df.Cover_Type
values = array(data)
print(values)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
type(onehot_encoded)

[5 5 2 ... 3 3 3]
[4 4 1 ... 2 2 2]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


numpy.ndarray

In [4]:
# make dataframe of binary outcomes for 7 cover types
data = onehot_encoded
dataset = pd.DataFrame({'bin_type1':data[:,0],'bin_type2':data[:,1],'bin_type3':data[:,2],'bin_type4':data[:,3],
                       'bin_type5':data[:,4],'bin_type6':data[:,5],'bin_type7':data[:,6]})
dataset.head()

,bin_type1,bin_type2,bin_type3,bin_type4,bin_type5,bin_type6,bin_type7
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
# combine dataframes to get binary outcome attached to cover types
# make sure they match
df = pd.concat([df, dataset], axis=1)
df.tail(50)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type39,Soil_Type40,Cover_Type,bin_type1,bin_type2,bin_type3,bin_type4,bin_type5,bin_type6,bin_type7
15070,15071,2523,210,30,190,119,1513,182,251,187,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15071,15072,2462,34,19,120,39,1597,214,196,112,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15072,15073,2487,197,24,240,-62,1273,207,252,168,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15073,15074,2434,225,35,30,23,1518,151,247,212,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15074,15075,2453,213,34,42,33,1509,170,248,193,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15075,15076,2430,143,22,256,60,1200,245,229,101,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15076,15077,2415,255,14,342,48,1110,187,248,199,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15077,15078,2524,34,22,30,4,1550,211,186,102,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15078,15079,2497,180,31,240,92,1224,215,240,135,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15079,15080,2420,167,23,190,50,1142,232,241,128,...,0,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
# check value counts are correct
bin_type_list = df.iloc[:, 56:].columns.tolist()
for bin_type in bin_type_list:
    print(df[bin_type].value_counts())

0.0    12960
1.0     2160
Name: bin_type1, dtype: int64
0.0    12960
1.0     2160
Name: bin_type2, dtype: int64
0.0    12960
1.0     2160
Name: bin_type3, dtype: int64
0.0    12960
1.0     2160
Name: bin_type4, dtype: int64
0.0    12960
1.0     2160
Name: bin_type5, dtype: int64
0.0    12960
1.0     2160
Name: bin_type6, dtype: int64
0.0    12960
1.0     2160
Name: bin_type7, dtype: int64


In [12]:
df.describe()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type39,Soil_Type40,Cover_Type,bin_type1,bin_type2,bin_type3,bin_type4,bin_type5,bin_type6,bin_type7
count,15120.00000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,...,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000,15120.000000
mean,7560.50000,2749.322553,156.676653,16.501587,227.195701,51.076521,1714.023214,212.704299,218.965608,135.091997,...,0.043452,0.030357,4.000000,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
std,4364.91237,417.678187,110.085801,8.453927,210.075296,61.239406,1325.066358,30.561287,22.801966,45.895189,...,0.203880,0.171574,2.000066,0.349939,0.349939,0.349939,0.349939,0.349939,0.349939,0.349939
min,1.00000,1863.000000,0.000000,0.000000,0.000000,-146.000000,0.000000,0.000000,99.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3780.75000,2376.000000,65.000000,10.000000,67.000000,5.000000,764.000000,196.000000,207.000000,106.000000,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7560.50000,2752.000000,126.000000,15.000000,180.000000,32.000000,1316.000000,220.000000,223.000000,138.000000,...,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,11340.25000,3104.000000,261.000000,22.000000,330.000000,79.000000,2270.000000,235.000000,235.000000,167.000000,...,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,15120.00000,3849.000000,360.000000,52.000000,1343.000000,554.000000,6890.000000,254.000000,254.000000,248.000000,...,1.000000,1.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# now split df into training and testing
# training will have validation set too
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df.Cover_Type)

In [14]:
print(df_train.Cover_Type.value_counts())
print(df_test.Cover_Type.value_counts())
# even splits

7    1728
6    1728
5    1728
4    1728
3    1728
2    1728
1    1728
Name: Cover_Type, dtype: int64
7    432
5    432
3    432
1    432
6    432
4    432
2    432
Name: Cover_Type, dtype: int64


In [15]:
df_test.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type39,Soil_Type40,Cover_Type,bin_type1,bin_type2,bin_type3,bin_type4,bin_type5,bin_type6,bin_type7
3551,3552,2311,102,25,525,17,1392,252,199,59,...,0,0,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5928,5929,2315,315,18,134,28,1500,170,223,192,...,0,0,6,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13057,13058,2276,91,16,0,0,1072,243,212,93,...,0,0,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2815,2816,2797,174,23,272,91,600,226,245,138,...,0,0,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12192,12193,2038,15,3,0,0,700,217,233,154,...,0,0,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
# function to run models and get evalution metrics
def run_models_scores(model, X_train, y_train, X_test, y_test):
    print(model) # know the model params
    start_time = time.time() 
    model.fit(X_train, y_train) # fit on training
    print("--- %s seconds ---" % (time.time() - start_time))
    y_pred = model.predict(X_test) # get prediction from testset
    # get eval metrics for test set
    print(classification_report(y_test, y_pred)) 
    print(confusion_matrix(y_test, y_pred))
    return (accuracy_score(y_test, y_pred))
# gets only eval metrics
def classreport_confmatrix_score(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    return (accuracy_score(y_test, y_pred))

In [17]:
# step 1 - prep to train boosted tree on binary outcomes
# X has no info about cover_type or binary categories
X = df_train.drop(['Cover_Type', 'Id', 'bin_type1', 'bin_type2', 'bin_type3',
                  'bin_type4', 'bin_type5', 'bin_type6', 'bin_type7'], 1)

y1 = df_train['bin_type1']
y2 = df_train['bin_type2']
y3 = df_train['bin_type3']
y4 = df_train['bin_type4']
y5 = df_train['bin_type5']
y6 = df_train['bin_type6']
y7 = df_train['bin_type7']

In [18]:
X.head() # just cartographic info and soil types and wilderness areas

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
14128,2942,151,27,376,191,1928,242,227,96,837,...,0,0,0,0,0,0,0,0,0,0
4425,2869,299,9,90,-11,1670,196,237,181,1460,...,0,0,0,0,0,0,0,0,0,0
909,2928,294,6,216,21,5075,204,239,174,1744,...,0,0,0,0,0,0,0,0,0,0
5123,2827,144,18,120,24,2255,242,234,114,1181,...,0,0,0,0,0,0,0,0,0,0
10901,2559,336,14,30,12,2006,187,220,171,524,...,0,0,0,0,0,0,0,0,0,0


In [19]:
X.shape # does not include cover type, bin types, or id

(12096, 54)

In [20]:
# ys are binary outcomes
y1

14128    0.0
4425     0.0
909      0.0
5123     0.0
10901    0.0
514      1.0
6952     0.0
7615     0.0
14101    0.0
5125     0.0
1696     0.0
9821     1.0
3116     0.0
11820    0.0
2709     0.0
2888     0.0
8385     1.0
13325    0.0
13483    0.0
12230    0.0
2384     0.0
14565    0.0
2001     0.0
14521    1.0
12131    0.0
2395     0.0
7382     1.0
12274    0.0
8485     0.0
14329    1.0
        ... 
2096     0.0
3310     0.0
8282     1.0
12604    0.0
1634     0.0
320      0.0
10375    0.0
10923    0.0
4711     0.0
14240    0.0
4107     0.0
7041     1.0
11064    0.0
14248    0.0
7306     0.0
8044     0.0
5667     0.0
7494     0.0
7709     0.0
435      0.0
997      0.0
11369    1.0
14108    1.0
6179     0.0
13936    0.0
5297     0.0
12296    0.0
8010     0.0
7557     0.0
11291    0.0
Name: bin_type1, Length: 12096, dtype: float64

In [21]:
# step 2
# build first stack of stacked model

params = {'n_estimators': 100,
          'max_depth': 12,
          'loss': 'exponential',
          'max_features': 4,
          'max_leaf_nodes': 500}

# 7 binary classifier models
clf1 = ensemble.GradientBoostingClassifier(**params)
clf2 = ensemble.GradientBoostingClassifier(**params)
clf3 = ensemble.GradientBoostingClassifier(**params)
clf4 = ensemble.GradientBoostingClassifier(**params)
clf5 = ensemble.GradientBoostingClassifier(**params)
clf6 = ensemble.GradientBoostingClassifier(**params)
clf7 = ensemble.GradientBoostingClassifier(**params)

model1 = clf1.fit(X, y1)
model2 = clf2.fit(X, y2)
model3 = clf3.fit(X, y3)
model4 = clf4.fit(X, y4)
model5 = clf5.fit(X, y5)
model6 = clf6.fit(X, y6)
model7 = clf7.fit(X, y7)

In [22]:
# step 3 get predictions and add to df_train from binary classifiers 
df_train['predict1'] = model1.predict(X)
df_train['predict2'] = model2.predict(X)
df_train['predict3'] = model3.predict(X)
df_train['predict4'] = model4.predict(X)
df_train['predict5'] = model5.predict(X)
df_train['predict6'] = model6.predict(X)
df_train['predict7'] = model7.predict(X)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [23]:
df_train.head(20)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,bin_type5,bin_type6,bin_type7,predict1,predict2,predict3,predict4,predict5,predict6,predict7
14128,14129,2942,151,27,376,191,1928,242,227,96,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4425,4426,2869,299,9,90,-11,1670,196,237,181,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
909,910,2928,294,6,216,21,5075,204,239,174,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5123,5124,2827,144,18,120,24,2255,242,234,114,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10901,10902,2559,336,14,30,12,2006,187,220,171,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
514,515,2995,62,11,484,17,3209,230,216,118,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6952,6953,2682,73,11,0,0,1458,233,218,115,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7615,7616,2671,45,13,85,18,2043,223,210,119,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
14101,14102,3323,5,11,525,89,721,206,219,150,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5125,5126,2361,329,31,95,43,170,132,187,185,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [28]:
# use function from above to get scores
binary_model_list = [model1, model2, model3, model4, model5, model6, model7]
ys = [y1, y2, y3, y4, y5, y6, y7]
for model_i, y_i in zip(binary_model_list, ys):
    print(classreport_confmatrix_score(model_i, X, y_i))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     10368
         1.0       1.00      1.00      1.00      1728

   micro avg       1.00      1.00      1.00     12096
   macro avg       1.00      1.00      1.00     12096
weighted avg       1.00      1.00      1.00     12096

[[10368     0]
 [    3  1725]]
0.9997519841269841
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     10368
         1.0       1.00      0.99      1.00      1728

   micro avg       1.00      1.00      1.00     12096
   macro avg       1.00      1.00      1.00     12096
weighted avg       1.00      1.00      1.00     12096

[[10368     0]
 [   14  1714]]
0.9988425925925926
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     10368
         1.0       1.00      1.00      1.00      1728

   micro avg       1.00      1.00      1.00     12096
   macro avg       1.00     

In [29]:
# final step in training
# look at random forest
X = df_train.drop(['Cover_Type', 'Id', 'bin_type1', 'bin_type2', 'bin_type3',
                  'bin_type4', 'bin_type5', 'bin_type6', 'bin_type7'], 1)
y = df_train['Cover_Type']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rfc = ensemble.RandomForestClassifier(criterion='entropy', 
                                      n_estimators=15,
                                      n_jobs=-1,
                                      max_depth=2
                                     )
# using function from beginning to run model and get scores
print(run_models_scores(rfc, X_train, y_train, X_val, y_val))
print()
print('training')
scores = cross_val_score(rfc, X_train, y_train, cv=10)
print(scores, scores.mean(), scores.std())
print('validating')
scores = cross_val_score(rfc, X_val, y_val, cv=10)
print(scores, scores.mean(), scores.std())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
--- 0.12607693672180176 seconds ---
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       345
           2       0.96      0.95      0.96       345
           3       1.00      0.98      0.99       346
           4       1.00      1.00      1.00       346
           5       0.99      1.00      1.00       346
           6       0.95      0.97      0.96       346
           7       1.00      1.00      1.00       346

   micro avg       0.99      0.99      0.99      2420
   macro avg       0.99      0.99      0.99      2420


In [30]:
# can we obtain good results on test set?

# step 1 - prep to get predictions

X = df_test.drop(['Cover_Type', 'Id', 'bin_type1', 'bin_type2', 'bin_type3',
                  'bin_type4', 'bin_type5', 'bin_type6', 'bin_type7'], 1)

y1 = df_test['bin_type1']
y2 = df_test['bin_type2']
y3 = df_test['bin_type3']
y4 = df_test['bin_type4']
y5 = df_test['bin_type5']
y6 = df_test['bin_type6']
y7 = df_test['bin_type7']

# step 2 build binary classifier models
model1 = clf1.fit(X, y1)
model2 = clf2.fit(X, y2)
model3 = clf3.fit(X, y3)
model4 = clf4.fit(X, y4)
model5 = clf5.fit(X, y5)
model6 = clf6.fit(X, y6)
model7 = clf7.fit(X, y7)

# step 3 get predictions into df_test
df_test['predict1'] = model1.predict(X)
df_test['predict2'] = model2.predict(X)
df_test['predict3'] = model3.predict(X)
df_test['predict4'] = model4.predict(X)
df_test['predict5'] = model5.predict(X)
df_test['predict6'] = model6.predict(X)
df_test['predict7'] = model7.predict(X)

# last last step if it works
# look at random forest

Xrf = df_test.drop(['Cover_Type', 'Id', 'bin_type1', 'bin_type2', 'bin_type3',
                  'bin_type4', 'bin_type5', 'bin_type6', 'bin_type7'], 1)
yrf = df_test['Cover_Type']

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# same rf model used on training set

print(rfc)
print('testing!')
scores = cross_val_score(rfc, Xrf, yrf, cv=10)
print(scores, scores.mean(), scores.std())

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
testing!
[0.93831169 0.83441558 0.90697674 0.99003322 0.99667774 0.80398671
 0.98671096 0.99667774 0.97674419 0.99335548] 0.9423890063424947 0.06781659726345739


In [31]:
classreport_confmatrix_score(rfc, Xrf, yrf)

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       432
           2       0.93      0.94      0.94       432
           3       1.00      0.96      0.98       432
           4       0.99      1.00      1.00       432
           5       1.00      1.00      1.00       432
           6       0.95      0.97      0.96       432
           7       1.00      1.00      1.00       432

   micro avg       0.98      0.98      0.98      3024
   macro avg       0.98      0.98      0.98      3024
weighted avg       0.98      0.98      0.98      3024

[[432   0   0   0   0   0   0]
 [  0 408   0   4   0  20   0]
 [  0  16 414   0   2   0   0]
 [  0   0   0 432   0   0   0]
 [  0   0   0   0 432   0   0]
 [  0  15   0   0   0 417   0]
 [  0   0   0   0   0   0 432]]


0.9811507936507936